# 概述

[Baidu ElasticSearch](https://cloud.baidu.com/doc/BES/index.html?from=productToDoc)（以下简称 BES）是一款100%兼容开源的分布式检索分析服务。为结构化/非结构化数据提供低成本、高性能及可靠性的检索、分析平台级产品服务。向量能力方面，支持多种索引类型和相似度距离算法。
目前，BES已经和文心一言大模型联合打造方案，也广泛应用于推荐、计算机视觉、智能问答等领域；同时，BES的向量检索和存储的能力也集成进入Langchain。
本文主要介绍基于Langchain的框架，结合BES的向量数据库的能力，对接千帆平台的模型管理和应用接入的能力，从而构建一个RAG的知识问答场景。

In [ ]:
bes,1
faiss,1
chroma, 1

postgresql
milvus, 
pincone2

## 准备工作
用户需要保证python的版本大于等于3.9，且需要安装langchain，qianfan sdk的包。

In [ ]:
#!pip install langchain==0.332
#!pip install elasticsearch == 7.11.0
#!pip install qianfan 

## 系统设置
1. 需要在千帆大模型平台创建应用，获得接入的ak、sk
2. 在BES的产品界面上创建一个BES集群，详见(https://cloud.baidu.com/doc/BES/s/0jwvyk4tv)

In [ ]:
import os

os.environ['QIANFAN_AK'] = "your_qianfan_ak"
os.environ['QIANFAN_SK'] = "your_qianfan_sk"

## 开发过程
### 文档加载、切分

In [ ]:
from langchain.document_loaders import TextLoader

loader = TextLoader("example_data/ai-paper.pdf")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=768, chunk_overlap=0, separators=["\n\n", "\n", " ", "", "。", "，"]) # spaciy
docs = text_splitter.split_documents(documents)

### 嵌入
创建对应的嵌入算法，这里采用千帆大模型平台的 `QianfanEmbeddingsEndpoint` 接口。

In [ ]:
from langchain.embeddings import QianfanEmbeddingsEndpoint #sdk

embeddings = QianfanEmbeddingsEndpoint()
# embeddings-v1
# bge-large-zh
# 12月2k
# 

### 向量检索和存储
引入创建的BES集群，用Langchain内部集成的`BESVectorStore`接口创建向量存储对象(详见:[BESVectorStore](https://python.langchain.com/docs/integrations/vectorstores/baiducloud_vector_search))

In [ ]:
from langchain.vectorstores import BESVectorStore

bes = BESVectorStore.from_documents(
    documents=docs,
    embedding=embeddings,
    bes_url="your bes cluster url",
    index_name="your vector index",
)
bes.client.indices.refresh(index="your vector index")

In [ ]:
#Faiss


### 生成QA检索器
创建对应 ERNIE-Bot 或者 ERNIE-Bot-turbo 模型的 Langchain Chat Model用于进一步生成QA检索器。model 字段支持使用 `ERNIE-Bot-turbo` 或 `ERNIE-Bot`。这里我们使用 `ERNIE-Bot`

In [ ]:
from langchain.chat_models import QianfanChatEndpoint

qianfan_chat_model = QianfanChatEndpoint(model="ERNIE-Bot")
# sdk prompt load from qianfan
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="refine", retriever=retriever, return_source_documents=True)


query = input("\n请输入问题: ")
res = qa(query)
answer, docs = res['result'], res['source_documents']

print("\n\n> 问题:")
print(query)
print("\n> 回答:")
print(answer)

In [ ]:
> 问题:
2021年12月,DeepMind做了什么?

> 回答:
根据上下文，2021年12月，DeepMind发布了Gopher模型。该模型具有2800亿参数。经过152个任务的评估，Gopher比当时最先进的语言模型提高了大约81%的性能，特别是在知识密集领域，如事实检测和常识上。
因此，正确答案是发布Gopher模型。